In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os, glob
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.models import Model, Sequential, Input, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D,Activation
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [33]:
train_dir=r"D:/PROJECTS/2022-2023/November/Business/Medicinalplants/app/Data/"

In [34]:
img_height,img_width=128,128
batch_size=32

In [35]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.3)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,                                            
                                                class_mode='categorical',
                                                    subset='training')
test_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                  subset='validation')

Found 2294 images belonging to 10 classes.
Found 978 images belonging to 10 classes.


In [36]:
print(train_generator.dtype)

float32


In [37]:
base_model = tf.keras.applications.MobileNet(input_shape=(img_height,img_width, 3), include_top=False,
                          weights='imagenet')
model1 = Sequential()
model1.add(base_model)
model1.add(GlobalAveragePooling2D())
model1.add(Dense(64, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.2))
model1.add(Dense(10, activation='sigmoid'))
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                65600     
_________________________________________________________________
batch_normalization_5 (Batch (None, 64)                256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 3,295,370
Trainable params: 3,273,354
Non-trainable params: 22,016
_______________________________________

In [38]:
model1.compile(optimizer='Adam',loss="categorical_crossentropy",metrics=["accuracy"])
hist1=model1.fit_generator(train_generator,epochs=10,validation_data=test_generator,verbose=1)
model1.save("models/Mobilenet.h5")

Epoch 1/10
72/72 [==============================] - 129s 2s/step - loss: 0.1833 - accuracy: 0.9507 - val_loss: 1.9369 - val_accuracy: 0.6166
Epoch 2/10
72/72 [==============================] - 103s 1s/step - loss: 0.0186 - accuracy: 0.9974 - val_loss: 0.1501 - val_accuracy: 0.9519
Epoch 3/10
72/72 [==============================] - 104s 1s/step - loss: 0.0196 - accuracy: 0.9956 - val_loss: 0.6656 - val_accuracy: 0.7904
Epoch 4/10
72/72 [==============================] - 103s 1s/step - loss: 0.0082 - accuracy: 0.9996 - val_loss: 0.1587 - val_accuracy: 0.9652
Epoch 5/10
72/72 [==============================] - 103s 1s/step - loss: 0.0121 - accuracy: 0.9974 - val_loss: 0.5306 - val_accuracy: 0.8661
Epoch 6/10
72/72 [==============================] - 108s 1s/step - loss: 0.0422 - accuracy: 0.9874 - val_loss: 0.8774 - val_accuracy: 0.7751
Epoch 7/10
72/72 [==============================] - 119s 2s/step - loss: 0.0546 - accuracy: 0.9852 - val_loss: 0.7648 - val_accuracy: 0.8272
Epoch 8/10
72